# TKinter项目实战-屏保
### 项目分析
- 屏保可以自己启动，也可以手动启动
- 一旦敲击键盘或者移动鼠标，或者其他的引发事件，就停止
- 如果屏保是一幅画，则没有画框
- 图像的动作是随机的，具有随机性。可能数量，颜色，大小，运动方向，变形等
    - 整个世界的构成是：
        - 球（大小，运动方向，位置，运动距离）balls
            - 数量，颜色，大小，运动方向，变形随机
            - 能动，能调用
        - 范围（边界）ScreenSaver
            - canvas（帆布），尺寸等于屏幕，没有边框，颜色单一

In [7]:
l = [1,2,34,5]
l[:1]

[1]

In [17]:
import tkinter
import random

class RandomBall():
    '''
    定义运动求的类
    '''
    # canvas:画布，所有东西在画布上呈现，屏幕宽高
    def __init__(self,canvas,screen_width,screen_height):
        # 球的初始位置要随机
        # xpos表示x坐标
        self.xpos = random.randint(10, int(screen_width)-10)
        self.ypos = random.randint(10, int(screen_height)-10)
        
        # 定义球的运动速度
        # 模拟运动，不断擦掉原来的画面，在新的地方刷新，同时保证不能每次运动范围太大
        self.xvelocity = random.randint(4,20)
        self.yvelocity = random.randint(4,20)
        
        
        # 定义屏幕的大小
        self.screenwidth = screen_width
        self.screenheight = screen_height
        
        # 定义球的大小
        # 此处球的大小用半径表示
        self.radius = random.randint(20,100)
        
        # 判断并调整起始位置, 避免因为初始位置在屏幕外, 导致球在边缘反复横跳
        if self.xpos + self.radius >= self.screenwidth:
            self.xpos = self.screenwidth - self.radius
        if self.xpos - self.radius <= 0:
            self.xpos = self.radius
        if self.ypos + self.radius >= self.screenheight:
            self.ypos = self.screenheight - self.radius
        if self.ypos - self.radius <= 0:
            self.ypos = self.radius
               
        # 定义颜色
        # RGB（0-255），CMYK（品红靛青黄黑颜料），HSB，LAB
        # 此处用lambda表达式
        c = lambda: random.randint(0,255)
        self.color = '#%02x%02x%02x'%(c(),c(),c())
        
        self.canvas = canvas
        
        self.x = screen_width
        self.y = screen_height  # 老师这个命名方式非常不好,容易引起误会, 以后得注意
        
    def create_ball(self):
        '''
        用构造函数定义的变量值，在canvas上画一个球
        '''
        # 画圆形，不需要定义圆心
        # tkInter没有画圆形函数，只有画椭圆，需要两个坐标
        # 在一个长方形内画椭圆我们只需要定义长方形左上角和右下角就好
        # 求两个坐标的方法：
        x1 = self.xpos - self.radius
        y1 = self.ypos - self.radius

        x2 = self.xpos + self.radius
        y2 = self.ypos + self.radius

        # 再有两个对角的坐标之后，可以画椭圆了
        # fill 表示填充颜色
        # ouline是边界的颜色
        self.item = self.canvas.create_oval(x1,y1,x2,y2,\
                                          fill=self.color,\
                                          outline=self.color)

    # 让球动起来
    def move_ball(self):
        # 移动球的时候，需要控制球的移动方向
        # 每次移动后，球都有一个新的坐标
        self.xpos += self.xvelocity
        self.ypos += self.yvelocity

        # 以下判断是否会撞墙
        if self.xpos > self.x - self.radius:
            self.xvelocity *= -1
        if self.xpos < 0 + self.radius:
            self.xvelocity *= -1 
        if self.ypos > self.y - self.radius:
            self.yvelocity *= -1
        if self.ypos < 0 + self.radius:
            self.yvelocity *= -1 

        # 在画布上挪动ball
        self.canvas.move(self.item, self.xvelocity, self.yvelocity)
        
    
class ScreenSaver():
    '''
    可以被启动
    '''
    # 如何装随机产生的球
    balls = list()
    
    def __init__(self):
        self.num_balls = random.randint(8,20)
        
        self.root = tkinter.Tk()
        
        # 取消边框
        self.root.overrideredirect(1)
        
        # 任何移动鼠标都应该退出屏保
        self.root.bind('<Motion>', self.myquit)
        # 同理键盘
        self.root.bind('<Key>', self.myquit)

        # 得到屏幕大小，规格
        w,h = self.root.winfo_screenwidth(),self.root.winfo_screenheight()
        
#         w,h = 1920, 1080
        # 创建画布，包括画布的归属，规格
        self.canvas = tkinter.Canvas(self.root, width=w, height=h)
        self.canvas.pack()
        
        # 在画布上画球
        for i in range(self.num_balls):
            ball = RandomBall(self.canvas, w, h)
            ball.create_ball()
            self.balls.append(ball)
            
        self.running_balls()
        self.root.mainloop()

    # 这是递归来写的...能不能改成while True来写呢, 能不能改成异步呢,下个cell试验一下
    def running_balls(self):
        for ball in self.balls:
            ball.move_ball()
        
        # after是200毫秒后启动一个函数，需要启动的函数是第二个参数
        self.canvas.after(100,self.running_balls)
        
    def myquit(self,event):
        # 此处只是利用了事件处理机制
        # 实际上并不关心事件的类型
        # 作业：
            # 次屏保程序扩展成，一旦捕获事件，则判断退出
            # 显示一个button，button上显示事件类型，点击button后屏保才退出
        self.root.destroy()
        


if __name__ == '__main__':        
    ScreenSaver()

In [8]:
l = [1,2,3,4,4]
l.insert(0,'x')
print(l)

['x', 1, 2, 3, 4, 4]


In [16]:
l = [(1,2), (8,9)]
print(l)
print(*l)

[(1, 2), (8, 9)]
(1, 2) (8, 9)


In [13]:
import tkinter
help(tkinter.Canvas.create_line)


Help on function create_line in module tkinter:

create_line(self, *args, **kw)
    Create line with coordinates x1,y1,...,xn,yn.



In [21]:
'''
在这个单元中, 我试图对这个屏保程序实现异步IO的应用
'''

import tkinter
import random
import asyncio
import time

class RandomBall():

    def __init__(self,canvas,screen_width,screen_height):

        self.xpos = random.randint(10, int(screen_width)-10)
        self.ypos = random.randint(10, int(screen_height)-10)

        self.xvelocity = random.randint(4,20)
        self.yvelocity = random.randint(4,20)

        self.screenwidth = screen_width
        self.screenheight = screen_height

        self.radius = random.randint(20,100)

        if self.xpos + self.radius >= self.screenwidth:
            self.xpos = self.screenwidth - self.radius
        if self.xpos - self.radius <= 0:
            self.xpos = self.radius
        if self.ypos + self.radius >= self.screenheight:
            self.ypos = self.screenheight - self.radius
        if self.ypos - self.radius <= 0:
            self.ypos = self.radius

        c = lambda: random.randint(0,255)
        self.color = '#%02x%02x%02x'%(c(),c(),c())
        
        self.canvas = canvas
        
        self.x = screen_width
        self.y = screen_height  # 老师这个命名方式非常不好,容易引起误会, 以后得注意
        
    def create_ball(self):

        x1 = self.xpos - self.radius
        y1 = self.ypos - self.radius

        x2 = self.xpos + self.radius
        y2 = self.ypos + self.radius

        self.item = self.canvas.create_oval(x1,y1,x2,y2,\
                                          fill=self.color,\
                                          outline=self.color)

    def move_ball(self):

        self.xpos += self.xvelocity
        self.ypos += self.yvelocity

        if self.xpos > self.x - self.radius:
            self.xvelocity *= -1
        if self.xpos < 0 + self.radius:
            self.xvelocity *= -1 
        if self.ypos > self.y - self.radius:
            self.yvelocity *= -1
        if self.ypos < 0 + self.radius:
            self.yvelocity *= -1 

        self.canvas.move(self.item, self.xvelocity, self.yvelocity)
        
    
class ScreenSaver():

    balls = list()
    
    def __init__(self):  # 实际上这个就是主程序, 可以当成__main__来看待
        self.num_balls = random.randint(8,20)
        
        self.root = tkinter.Tk()
        
        self.root.overrideredirect(1)
        
        self.root.bind('<Motion>', self.myquit)

        self.root.bind('<Key>', self.myquit)

        w,h = self.root.winfo_screenwidth(),self.root.winfo_screenheight()
        
        self.canvas = tkinter.Canvas(self.root, width=w, height=h)
        self.canvas.pack()
        
        for i in range(self.num_balls):
            ball = RandomBall(self.canvas, w, h)
            ball.create_ball()
            self.balls.append(ball)
            
        loop_ = asyncio.get_event_loop()
        tasks = []
        for ball in self.balls:
            tasks.append(self.running_balls(ball))
        loop_.run_until_complete(asyncio.wait(tasks))
        loop_.close()
        
        # 跑到这里果然出了问题, 在主循环创建之前,程序就卡在await那里了.
        self.root.mainloop()
        

    async def running_balls(self, ball):
        ball.move_ball()
        await time.sleep(0.04)

    def myquit(self,event):
        # 此处只是利用了事件处理机制
        # 实际上并不关心事件的类型
        # 作业：
            # 次屏保程序扩展成，一旦捕获事件，则判断退出
            # 显示一个button，button上显示事件类型，点击button后屏保才退出
        self.root.destroy()


if __name__ == '__main__':        
    ScreenSaver()

RuntimeError: This event loop is already running